# Question 1

<div style="font-size:16px">
$\text{Gating Network Output} = \displaystyle \begin{bmatrix}A & A & A & B & A & A & A & A & A & B\end{bmatrix}$<br>
$\text{MoE} = \displaystyle \begin{bmatrix}0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 1\end{bmatrix}$<br>
$P_{\text{Misclassification}} = \displaystyle \frac{1}{10} = 0.1$
</div>

In [1]:
# Question 2

import numpy as np
from sklearn.model_selection import train_test_split

filename = 'winequality-white.csv'
data = np.genfromtxt(filename, dtype='f4', delimiter=',')

X = data[1:, :-1]
y = data[1:,  -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=None)

In [2]:
mean_scaled = np.mean(X_train, axis=0)
std_scaled = np.std(X_train, axis=0)

def normalize(X: np.ndarray) -> np.ndarray:
    return (X - mean_scaled) / std_scaled

X_train = normalize(X_train)
X_test = normalize(X_test)

In [3]:
# Gating Network initiation: Using kmeans algorithm to partition the data into 3 clusters
from sklearn.cluster import KMeans

C = 7
kmeans = KMeans(n_clusters=C, random_state=0).fit(X_train)
idx = kmeans.labels_

In [4]:
# Traing the gating network model using a naive baysian classifier using the clustering indices obtained after clustering
from sklearn.naive_bayes import GaussianNB

gNet = GaussianNB().fit(X_train, idx)

In [5]:
# Traing individual expert classifiers using each cluster of training data
from sklearn.tree import DecisionTreeClassifier

Exp = []
for k in range(C):
    Exp.append(DecisionTreeClassifier(random_state=0).fit(X_train[idx==k], y_train[idx==k]))

In [6]:
# Validation with X_test, y_test
from keras.utils.np_utils import to_categorical   

# Gating network output
y_gNet = gNet.predict(X_test)
y_gNet_1h = to_categorical(y_gNet)


# Let each expert predict for the entire validation data set
y_exp = []
for k in range(C):
    y_exp.append(Exp[k].predict(X_test))

# Selecting the prediction from corresponding expert assigned by the gating network
y_exp_T = np.array(y_exp).T
y_gated = np.sum(y_exp_T * y_gNet_1h, axis=1)

In [7]:
# Check results
from sklearn.metrics import confusion_matrix

# expert classifier's performance w.r.t. the entire validation dataset
for k in range(C):
    print(f'Expert #{k}\n',confusion_matrix(y_test,y_exp[k]))

print('Gated \n', confusion_matrix(y_test,y_gated))

Expert #0
 [[  1   0   5   5   0   0]
 [  0   1  26  14  11   0]
 [  0  28 210 148  49   1]
 [  0  54 210 248 152   1]
 [  0  15 106  69  63   2]
 [  0   2  20   8  19   2]]
Expert #1
 [[  0   0   3   8   0   0]
 [  0   1   6  44   1   0]
 [  0   2 112 302  20   0]
 [  0   3  83 544  35   0]
 [  0   0  17 227  11   0]
 [  0   0   1  46   4   0]]
Expert #2
 [[  0   1   5   4   1   0   0]
 [  0  13  17  11  11   0   0]
 [  0  27 220 124  59   4   2]
 [  0  38 112 277 225  12   1]
 [  0  15  21  43 169   7   0]
 [  0   3   4   7  31   6   0]
 [  0   0   0   0   0   0   0]]
Expert #3
 [[  1   1   2   4   3   0   0]
 [  2   7  16  25   2   0   0]
 [  4  15 120 243  49   5   0]
 [  1  11 149 345 127  21  11]
 [  0   1  43  98  89  13  11]
 [  0   1  11  16  16   5   2]
 [  0   0   0   0   0   0   0]]
Expert #4
 [[  0   2   3   6   0   0]
 [  0  16  18  15   1   2]
 [  0  68 173 168  16  11]
 [  0  79 181 248 100  57]
 [  0  27  31  75  72  50]
 [  0   4   5  23  13   6]]
Expert #5
 [[  0   1

In [8]:
# find which expert made the wrong classification
print('expert #',y_gNet[y_gated != y_test],'made incorrect prediction')

expert # [6 6 2 3 6 2 3 0 6 2 6 6 3 2 6 0 2 6 0 3 2 2 0 4 2 4 6 2 1 6 2 3 3 6 2 6 1
 6 2 2 6 0 5 3 2 6 3 4 6 2 6 6 0 6 6 3 5 6 2 6 2 6 6 6 3 6 3 6 2 6 2 6 6 6
 6 6 6 5 0 4 2 0 6 2 0 5 6 5 6 6 6 4 6 6 3 2 6 3 2 6 5 6 5 2 5 6 6 3 2 0 6
 6 2 2 6 6 0 6 0 6 3 6 6 6 3 6 3 2 6 0 0 4 4 6 6 6 3 4 4 6 2 6 2 3 6 4 0 3
 3 3 3 2 0 2 5 0 6 2 6 6 6 6 6 0 4 5 6 2 6 6 5 4 2 0 4 6 6 5 6 2 0 3 0 3 3
 2 3 3 6 5 2 6 2 3 6 6 3 0 6 2 5 2 5 3 6 0 0 3 6 0 0 5 2 6 6 6 6 3 0 6 5 4
 6 0 6 5 4 4 5 6 5 6 3 2 2 2 4 3 2 6 6 6 6 5 3 2 6 3 2 5 4 2 2 2 3 5 6 3 3
 6 6 6 0 2 6 3 2 3 5 2 3 2 5 6 5 2 6 3 2 6 5 0 3 3 3 4 5 1 5 6 0 3 3 2 2 2
 3 3 6 6 5 2 2 6 2 1 6 6 6 5 6 6 3 2 0 6 3 6 2 2 6 6 3 4 6 5 5 3 6 6 1 6 6
 2 2 3 4 3 2 5 6 3 3 6 3 3 3 6 5 6 6 6 3 0 2 3 6 6 6 2 6 5 3 6 3 2 5 6 2 2
 2 3 3 5 2 0 6 2 4 2 4 3 2 3 2 3 5 5 6 3 3 2 4 2 3 6 6 6 6 3 2 6 3 6 6 2 3
 0 5 3 6 6 0 6 1 5 4 5 2 2 6 0 5 2 5 4 0 2 6 6 3 3 6 2 1 4 0 2 3 2 2 0 6 6
 4 6 4 5 4 2 3 3 0 6 5 2 3 6 0 4 3 6 3 6 3 6 0 5 5 2 2 1 0 6 2 2 6 3 4 3 4
 2 3 6 2 6 6 5 6